---
## 1️⃣ Data Loading & Initial Overview

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries loaded successfully!")

In [ ]:
# Load dataset from parent directory
data_path = Path('../Assignment 2/crime_district.csv')
df = pd.read_csv(data_path)

print(f"📊 Dataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Display first few rows
print("\n🔍 First 10 rows:")
df.head(10)

In [ ]:
# Basic info
print("\n📋 Dataset Info:")
df.info()

In [ ]:
# Column names and types
print("\n📝 Columns:")
for col in df.columns:
    print(f"  • {col}: {df[col].dtype}")

---
## 2️⃣ Structure Analysis

In [ ]:
# Unique values per column
print("\n🔢 Unique Values Count:")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"  • {col}: {unique_count:,}")

In [ ]:
# Unique states
print("\n🗺️ Unique States:")
states = df['state'].unique()
print(f"Total: {len(states)}")
print(sorted(states))

In [ ]:
# Unique districts per state
print("\n🏘️ Districts per State:")
district_counts = df[df['district'] != 'All'].groupby('state')['district'].nunique().sort_values(ascending=False)
print(district_counts)

In [ ]:
# Crime categories and types
print("\n🚨 Crime Categories:")
print(df['category'].unique())

print("\n🔍 Crime Types:")
print(sorted(df['type'].unique()))

In [ ]:
# Hierarchical structure check
print("\n🌳 Hierarchical Structure Analysis:")
print(f"\nRows with 'Malaysia' as state: {len(df[df['state'] == 'Malaysia']):,}")
print(f"Rows with 'All' as district: {len(df[df['district'] == 'All']):,}")
print(f"\n⚠️ AGGREGATION WARNING: 'All' rows are TOTALS - will cause double-counting!")

---
## 3️⃣ Data Quality Assessment

In [ ]:
# Missing values
print("\n🔍 Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])

if missing.sum() == 0:
    print("✓ No missing values detected!")

In [ ]:
# Zero crimes analysis
print("\n🔍 Zero Crime Entries:")
zero_crimes = df[df['crimes'] == 0]
print(f"Total rows with 0 crimes: {len(zero_crimes):,} ({len(zero_crimes)/len(df)*100:.2f}%)")

if len(zero_crimes) > 0:
    print("\nStates/Districts with zeros:")
    print(zero_crimes.groupby(['state', 'district'])['crimes'].count().head(10))

In [ ]:
# Duplicate check
print("\n🔍 Duplicate Rows:")
duplicates = df.duplicated().sum()
print(f"Total duplicates: {duplicates}")

if duplicates > 0:
    print("\nDuplicate examples:")
    print(df[df.duplicated(keep=False)].head(10))

In [ ]:
# Statistical summary
print("\n📊 Crime Count Statistics:")
df['crimes'].describe()

---
## 4️⃣ Temporal Analysis

In [ ]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

print("\n📅 Temporal Coverage:")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Years covered: {sorted(df['year'].unique())}")
print(f"Total years: {df['year'].nunique()}")

In [ ]:
# Check if dates are all January 1st (yearly aggregates)
print("\n⚠️ Date Granularity Check:")
date_check = df['date'].dt.month.value_counts()
print(f"All dates are in month: {date_check.index.tolist()}")
day_check = df['date'].dt.day.value_counts()
print(f"All dates are on day: {day_check.index.tolist()}")
print("\n⚠️ WARNING: Data is YEARLY aggregates only (all dates = Jan 1st)")
print("   This limits time series forecasting capabilities!")

In [ ]:
# Yearly crime trend (excluding 'All' aggregates)
df_districts = df[df['district'] != 'All'].copy()

yearly_crimes = df_districts.groupby('year')['crimes'].sum().reset_index()

plt.figure(figsize=(12, 6))
plt.plot(yearly_crimes['year'], yearly_crimes['crimes'], marker='o', linewidth=2, markersize=8)
plt.title('📈 Total Crime Trend in Malaysia (2016-2023)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Crimes', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Yearly Crime Totals:")
print(yearly_crimes)

---
## 5️⃣ Crime Distribution Analysis

In [ ]:
# Crime by category (excluding aggregates)
category_crimes = df_districts.groupby('category')['crimes'].sum().sort_values(ascending=False)

print("\n🚨 Total Crimes by Category (2016-2023):")
print(category_crimes)

plt.figure(figsize=(10, 6))
category_crimes.plot(kind='bar', color=['#FF6B6B', '#4ECDC4'])
plt.title('🚨 Crime Distribution by Category', fontsize=16, fontweight='bold')
plt.xlabel('Crime Category', fontsize=12)
plt.ylabel('Total Crimes', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 crime types
type_crimes = df_districts[df_districts['type'] != 'all'].groupby('type')['crimes'].sum().sort_values(ascending=False).head(10)

print("\n🔝 Top 10 Crime Types:")
print(type_crimes)

plt.figure(figsize=(12, 6))
type_crimes.plot(kind='barh', color='#FF6B6B')
plt.title('🔝 Top 10 Crime Types in Malaysia', fontsize=16, fontweight='bold')
plt.xlabel('Total Crimes', fontsize=12)
plt.ylabel('Crime Type', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 high-crime states (excluding 'All')
state_crimes = df_districts[df_districts['state'] != 'Malaysia'].groupby('state')['crimes'].sum().sort_values(ascending=False).head(10)

print("\n🗺️ Top 10 States by Total Crime:")
print(state_crimes)

plt.figure(figsize=(12, 6))
state_crimes.plot(kind='barh', color='#4ECDC4')
plt.title('🗺️ Top 10 High-Crime States', fontsize=16, fontweight='bold')
plt.xlabel('Total Crimes (2016-2023)', fontsize=12)
plt.ylabel('State', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Top 15 high-crime districts
district_crimes = df_districts[df_districts['district'] != 'All'].groupby(['state', 'district'])['crimes'].sum().sort_values(ascending=False).head(15)

print("\n🏘️ Top 15 High-Crime Districts:")
print(district_crimes)

plt.figure(figsize=(12, 8))
district_crimes.plot(kind='barh', color='#FFA07A')
plt.title('🏘️ Top 15 High-Crime Districts in Malaysia', fontsize=16, fontweight='bold')
plt.xlabel('Total Crimes (2016-2023)', fontsize=12)
plt.ylabel('District (State)', fontsize=12)
plt.tight_layout()
plt.show()

---
## 6️⃣ Hierarchical Structure Mapping

In [ ]:
# Check if aggregates match district sums
print("\n🔍 Validation: Do state aggregates match district sums?\n")

# Example: Johor state in 2019, assault category
johor_state_total = df[(df['state'] == 'Johor') & 
                       (df['district'] == 'All') & 
                       (df['year'] == 2019) & 
                       (df['category'] == 'assault')]['crimes'].sum()

johor_districts_total = df[(df['state'] == 'Johor') & 
                           (df['district'] != 'All') & 
                           (df['year'] == 2019) & 
                           (df['category'] == 'assault') &
                           (df['type'] == 'all')]['crimes'].sum()

print(f"Johor State Total (2019, assault): {johor_state_total:,}")
print(f"Johor Districts Sum (2019, assault): {johor_districts_total:,}")
print(f"Match: {johor_state_total == johor_districts_total}")

if johor_state_total == johor_districts_total:
    print("\n✓ State aggregates = Sum of districts (hierarchy is correct!)")
else:
    print("\n⚠️ WARNING: State aggregates ≠ Sum of districts!")

In [ ]:
# Count rows by hierarchy level
print("\n📊 Row Distribution by Hierarchy Level:\n")

malaysia_level = len(df[df['state'] == 'Malaysia'])
state_level = len(df[(df['state'] != 'Malaysia') & (df['district'] == 'All')])
district_level = len(df[(df['state'] != 'Malaysia') & (df['district'] != 'All')])

print(f"National Level (Malaysia): {malaysia_level:,} rows")
print(f"State Level (All districts): {state_level:,} rows")
print(f"District Level (actual districts): {district_level:,} rows")
print(f"\nTotal: {malaysia_level + state_level + district_level:,} rows")

---
## 7️⃣ Key Findings & Recommendations

In [ ]:
# Summary of findings
print("""
╔════════════════════════════════════════════════════════════════════╗
║                  🚨 DATA QUALITY FINDINGS                          ║
╚════════════════════════════════════════════════════════════════════╝

✅ POSITIVE FINDINGS:
  • No missing values
  • Consistent date format
  • Clean hierarchical structure
  • 8 years of data (2016-2023)

⚠️ ISSUES IDENTIFIED:

1. HIERARCHICAL AGGREGATION
   - 'All' rows contain TOTALS (state/national level)
   - Using these will cause DOUBLE-COUNTING
   - Solution: Remove 'All' district rows for ML models

2. TEMPORAL GRANULARITY
   - Data is YEARLY aggregates only (all dates = Jan 1st)
   - Limited forecasting capability (only 8 data points)
   - Solution: Consider interpolation or quarterly estimates

3. MISSING DATA PATTERNS
   - Iskandar Puteri has 0 crimes in 2016
   - Likely new district (established 2017?)
   - Solution: Handle as valid 0 or exclude 2016 for this district

4. SPARSE CRIME TYPES
   - Some crime types very rare (robbery_solo_armed: single digits)
   - Will cause issues in classification models
   - Solution: Group rare crime types or exclude from models

╔════════════════════════════════════════════════════════════════════╗
║                  📋 ETL RECOMMENDATIONS                            ║
╚════════════════════════════════════════════════════════════════════╝

NEXT STEPS:

1. CREATE CLEANED DATASET
   ✓ Remove rows where district = 'All'
   ✓ Remove rows where state = 'Malaysia'
   ✓ Keep only district-level data

2. FEATURE ENGINEERING
   ✓ Create year-over-year change %
   ✓ Calculate crime rates (if population data available)
   ✓ Create crime diversity index per district
   ✓ Flag high-risk districts (threshold-based)

3. PREPARE ML DATASETS
   ✓ Time series: aggregate by district/state for forecasting
   ✓ Clustering: feature matrix (avg_assault, avg_property, etc.)
   ✓ Classification: district features → risk level

4. ADDRESS FORECASTING LIMITATION
   ✓ Consider using state-level trends
   ✓ Explore external data (economic indicators, weather)
   ✓ Use ensemble methods for limited time points
""")

---
## 🎯 Next Notebook: `02_ETL_Data_Preparation.ipynb`

**What we'll build:**
1. Data cleaning pipeline
2. Feature engineering functions
3. Export cleaned datasets for:
   - PowerBI dashboard
   - Time series forecasting
   - Clustering analysis
   - Classification models